# The `wrapt` Library

This is a super useful library for easily writing decorators in Python, with a whole slew of additional functionality.

If you find yourself writing a lot of decorators in Python (or maybe even just a few!), you ought to give this library a try. It's very easy to get started with it, and in the longer term you can increase your utilization of it with its more advanced functionality.

Let's see how we would create a simple decorator to maybe log function calls.

## Setting up Python's Logger

Instead of using print statements, we'll actually use Python's logger - but we just to set things up a little differently than you would with a regular Python app since Jupyter itself has already configured a logger.

First we create a custom handler:

In [1]:
import logging

In [2]:
handler = logging.StreamHandler()
formatter = logging.Formatter(
    fmt='{asctime} - {levelname} - {message}',
    datefmt="%Y-%m-%d %H:%M:%S",
    style="{"
)
handler.setFormatter(formatter)                              

If you're curious about the attributes I used in the formatter, you can find a complete list here:
[https://docs.python.org/3/library/logging.html#logrecord-attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes)

Next, we add this handler to the existing root handler:

In [3]:
logging.getLogger().addHandler(handler)

Finally, we need to get to the logger for this particular notebook, and we'll set the logging level too while we're at it:

In [4]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

Now we can see our logging in our notebook instead of just in the Jupyter log window (accessed via the `View` menu.)

In [5]:
logger.critical("this is a test")

2024-03-23 22:13:55 - CRITICAL - this is a test


## Standard Decorator Approach

In [6]:
from functools import wraps

def log(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        try:
            result = fn(*args, **kwargs)
            logger.info(f"called with: {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
            raise
        
    return inner

In [7]:
@log
def add(a, b):
    return a + b

In [8]:
add(1, 2)

2024-03-23 22:13:55 - INFO - called with: args=(1, 2), kwargs={}


3

In [9]:
try:
    add("a", 1)
except Exception as ex:
    print("Exception was raised:", type(ex), str(ex))

2024-03-23 22:13:55 - ERROR - called with: args=('a', 1), kwargs={}, ex=can only concatenate str (not "int") to str


Exception was raised: <class 'TypeError'> can only concatenate str (not "int") to str


Maybe, we want the option to silence exceptions, just log the exception, return None, and continue.

We can do that by creating a decorator that can take an argument to control that

In [10]:
def log(*, silence=False):
    def decorator(fn):
        @wraps(fn)
        def inner(*args, **kwargs):
            try:
                result = fn(*args, **kwargs)
                logger.info(f"called with: {args=}, {kwargs=}")
                return result
            except Exception as ex:
                logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
                if not silence:
                    raise
            
        return inner
    return decorator

In [11]:
@log()
def add(a, b):
    return a + b

In [12]:
add(1, 2)

2024-03-23 22:13:55 - INFO - called with: args=(1, 2), kwargs={}


3

In [13]:
try:
    add("a", 1)
except Exception as ex:
    print("Exception was raised:", type(ex), str(ex))

2024-03-23 22:13:55 - ERROR - called with: args=('a', 1), kwargs={}, ex=can only concatenate str (not "int") to str


Exception was raised: <class 'TypeError'> can only concatenate str (not "int") to str


And if we want it silenced:

In [14]:
@log(silence=True)
def add(a, b):
    return a + b

In [15]:
try:
    add("a", 1)
except Exception as ex:
    print("Exception was raised:", str(ex))

2024-03-23 22:13:55 - ERROR - called with: args=('a', 1), kwargs={}, ex=can only concatenate str (not "int") to str


## The `wrapt` Approach

In [16]:
import wrapt

Let's start by re-writing the first decorator we had:

In [17]:
def log(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        try:
            result = fn(*args, **kwargs)
            logger.info(f"called with: {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
            raise
        
    return inner

Using `wrapt`, we do this instead:

In [18]:
@wrapt.decorator
def log(fn, instance, args, kwargs):
    try:
        result = fn(*args, **kwargs)
        logger.info(f"called with: {args=}, {kwargs=}")
        return result
    except Exception as ex:
        logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
        raise

In [19]:
@log
def add(a, b):
    return a + b

In [20]:
add(1, 2)

2024-03-23 22:13:55 - INFO - called with: args=(1, 2), kwargs={}


3

In [21]:
try:
    add("a", 1)
except Exception as ex:
    print("Exception was raised:", type(ex), str(ex))

2024-03-23 22:13:55 - ERROR - called with: args=('a', 1), kwargs={}, ex=can only concatenate str (not "int") to str


Exception was raised: <class 'TypeError'> can only concatenate str (not "int") to str


Next, let's see how we implement the second version we wrote:

In [22]:
def log(*, silence=False):
    def decorator(fn):
        @wraps(fn)
        def inner(*args, **kwargs):
            try:
                result = fn(*args, **kwargs)
                logger.info(f"called with: {args=}, {kwargs=}")
                return result
            except Exception as ex:
                logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
                if not silence:
                    raise
            
        return inner
    return decorator

Using `wrapt` we can re-write it this way:

In [23]:
def log(*, silence=False):
    @wrapt.decorator
    def inner(fn, instance, args, kwargs):
        try:
            result = fn(*args, **kwargs)
            logger.info(f"called with: {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
            if not silence:
                raise

    return inner

In [24]:
@log()
def add(a, b):
    return a + b

In [25]:
try:
    add("a", 1)
except Exception as ex:
    print("Exception was raised:", type(ex), str(ex))

2024-03-23 22:13:55 - ERROR - called with: args=('a', 1), kwargs={}, ex=can only concatenate str (not "int") to str


Exception was raised: <class 'TypeError'> can only concatenate str (not "int") to str


In [26]:
@log(silence=True)
def add(a, b):
    return a + b

## Enabling and Disabling Decorators

We very well could have a scenario where we want to enable or disable our log decorator depending on circumstances.

Although we can do that in plain Python, it would take a little bit of work to do this, and we're not going to attempt it (though certainly give it a try on your own!)

Instead, let's use `wrapts` built-in functionality to do so.

Suppose, we have some module level variable (or we read it from some config) that sets a flag to enable or disable the logging decorator.

In [27]:
LOG_ENABLED = True

Then, when we define our decorator we can use the special argument `enabled` to set that state:

In [28]:
def log(*, silence=False):
    @wrapt.decorator(enabled=LOG_ENABLED)
    def inner(fn, instance, args, kwargs):
        try:
            result = fn(*args, **kwargs)
            logger.info(f"called with: {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
            if not silence:
                raise

    return inner

In [29]:
add(1, 2)

2024-03-23 22:13:55 - INFO - called with: args=(1, 2), kwargs={}


3

Now let's turn the functionality off. We can't just do this:

In [30]:
LOG_ENABLED = False

In [31]:
add(1, 2)

2024-03-23 22:13:55 - INFO - called with: args=(1, 2), kwargs={}


3

This approach only works when our application starts up, and the enabling/disabling of the decorator is essentially set the first time the code is compiled.

So, to mimic this here without restarting out Jupyter notebook, we can do this:

In [32]:
LOG_ENABLED = False

def log(*, silence=False):
    @wrapt.decorator(enabled=LOG_ENABLED)
    def inner(fn, instance, args, kwargs):
        try:
            result = fn(*args, **kwargs)
            logger.info(f"called with: {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
            if not silence:
                raise

    return inner

In [33]:
@log(silence=True)
def add(a, b):
    return a + b

In [34]:
add(1, 2)

3

So this approach works great, and is good as long as we want that enabled/disabled state to be set at app startup.

But what about when we want it to be dynamic while our application is running - i.e. we may want to turn the logger on sometimes, and turn it off other times.

Well, `wrapt` has that covered too!!! All we need to do is specify a **callable** that returns a boolean.

here, we'll just use a lambda that returns that global variable:

In [35]:
LOGGER_ENABLED = True

In [36]:
def log(*, silence=False):
    @wrapt.decorator(enabled=lambda: LOGGER_ENABLED)
    def inner(fn, instance, args, kwargs):
        try:
            result = fn(*args, **kwargs)
            logger.info(f"called with: {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called with: {args=}, {kwargs=}, ex={str(ex)}")
            if not silence:
                raise

    return inner

In [37]:
@log()
def add(a, b):
    return a + b

@log()
def mult(a, b):
    return a * b

In [38]:
add(1, 2)

2024-03-23 22:13:55 - INFO - called with: args=(1, 2), kwargs={}


3

In [39]:
mult(1, 2)

2024-03-23 22:13:55 - INFO - called with: args=(1, 2), kwargs={}


2

Now, let's change that global:

In [40]:
LOGGER_ENABLED = False

In [41]:
add(1, 2)

3

In [42]:
mult(1, 2)

2

You can think that you may have some decorators that need to be enabled or disabled based on the results of some function calling into a database, a "live" config, an API result, etc. Possibilities are endless and `wrapts` makes it super easy to do.

## Introspection

Under the hood, `wrapts` also provides much better support for introspection, far more than Python's `wraps` itself.

Let's go back to a basic Python decorator:

In [43]:
def my_dec(fn):
    def inner(*args, **kwargs):
        return fn(*args, **kwargs)

    return inner

In [44]:
@my_dec
def add(a, b, *, extras=None):
    """This functions adds two objects, as long as they support addition.
        extras: just for demonstrating something
    """
    return a + b

In [45]:
add(1, 2)

3

In [46]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



Hmm... We are getting the docs for the `inner` function, not the `add` function. As we know, we should use `@wraps` to fix that:

In [47]:
def my_dec(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        return fn(*args, **kwargs)

    return inner

In [48]:
@my_dec
def add(a, b, *, extras=None):
    """This functions adds two objects, as long as they support addition.
        extras: just for demonstrating something
    """
    return a + b

In [49]:
help(add)

Help on function add in module __main__:

add(a, b, *, extras=None)
    This functions adds two objects, as long as they support addition.
    extras: just for demonstrating something



Next, let's see how the introspection module `inspect` works with our decorated function:

In [50]:
import inspect

In [51]:
inspect.getdoc(add)

'This functions adds two objects, as long as they support addition.\nextras: just for demonstrating something'

That worked, but what about inspecting code and args?

In [52]:
print(inspect.getsource(add))

@my_dec
def add(a, b, *, extras=None):
    """This functions adds two objects, as long as they support addition.
        extras: just for demonstrating something
    """
    return a + b



In [53]:
inspect.getfullargspec(add)

FullArgSpec(args=[], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

As you can see, inspecting the args is clearly not returning the args for `add`, but rather for the `inner` function.

So, let's see how `wrapts` handles that:

In [54]:
@wrapt.decorator
def my_dec(fn, instance, args, kwargs):
    return fn(*args, **kwargs)
    

In [55]:
@my_dec
def add(a, b, *, extras=None):
    """This functions adds two objects, as long as they support addition.
        extras: just for demonstrating something
    """
    return a + b

In [56]:
add(1, 2)

3

In [57]:
help(add)

Help on function add in module __main__:

add(a, b, *, extras=None)
    This functions adds two objects, as long as they support addition.
    extras: just for demonstrating something



In [58]:
print(inspect.getsource(add))

@my_dec
def add(a, b, *, extras=None):
    """This functions adds two objects, as long as they support addition.
        extras: just for demonstrating something
    """
    return a + b



In [59]:
inspect.getfullargspec(add)

FullArgSpec(args=['a', 'b'], varargs=None, varkw=None, defaults=None, kwonlyargs=['extras'], kwonlydefaults={'extras': None}, annotations={})

As you can see, the inspection of args now works as well.

## Conclusion

The `wrapt` library has a number of other tricks up its sleeve, including straightforward ways to decorate class instance methods, class methods, static methods, and even decorating classes themselves.

So, give the `wrapt` library a try, and see how you like it!